In [2]:
"""
This notebook runs two different versions of finding all nearest
neighbors in a randomly generated set of points.

Change the dimensions of the points and size of data set to
see how the runtime is impacted
"""

import random
from sklearn.neighbors import NearestNeighbors
import numpy as np


def generate_points(n_points, dimensions):
    """Generates points
    
    Args:
        n_points (int): number of points to generate
        dimensions (int): number of dimensions each point lives in
        
    Returns:
        list
    """
    def gen_point(d):
        return [random.random() for i in range(d)]
    
    return [gen_point(dimensions) for i in range(n_points)]

def distance(p1, p2):
    return sum((i - j)**2 for (i, j) in zip(p1, p2))**0.5

In [3]:
# Naive Implementation O(n^2)
def naive_get_nearest_neighbor(all_points, start_point):
    """Find the nearest point the start_point inside of all_points
    
    Args:
        all_points (list of points): all available points for comparison
        start_point (vector): a single point to center the rest of data set around
        
    Return:
        vector, of the closest point
    """
    closest = all_points[0]
    d_closest = distance(closest, start_point)
    for point in all_points[1:]:
        d_candidate = distance(point, start_point)
        if d_candidate < d_closest:
            closest = point
            d_closest = d_candidate
            
    return closest


def naive_get_all_nn(data):
    return [naive_get_nearest_neighbor(data, point) for point in data]

In [4]:
# Hash points to buckets and then look for nearest neighbors
"""
Two known bugs, nearest neighbors are returned in a scrambled order - need an index to
straighten out. The hashing is not loss-less, it has trouble with boundary conditions.
"""
from collections import defaultdict
hash_vectors = generate_points(3, 10)

def compare(v1, v2):
    over_under = sum(1 if i > j else 0 for (i, j) in zip(v1, v2)) - 5
    return 1 if over_under > 0 else -1

def hash_point(point):
    return (compare(point, h) for h in hash_vectors)

def hashed_get_all_nn(data):
    buckets = defaultdict(list)
    for point in data:
        buckets[hash_point(point)].append(point)
        
    # Note this is slightly buggy as returned in in correct order
    return [naive_get_all_nn(points) for points in buckets.values()]

In [5]:
# Built in Nearest Neighbors
def built_in_get_all_nn(data):
    X = np.array(data)
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(X)
    return nbrs.kneighbors(X)[0]

In [14]:
# Make sure they all run - basic baby problem
data = generate_points(10**1, 10)

print('\nNaive')
%time naive_nn = naive_get_all_nn(data)

print('\nHashed')
%time hashed_nn = hashed_get_all_nn(data)

print('\nBuilt In')
%time built_in_nn = built_in_get_all_nn(data)


Naive
CPU times: user 925 µs, sys: 1.05 ms, total: 1.97 ms
Wall time: 2.11 ms

Hashed
CPU times: user 6.03 ms, sys: 17.3 ms, total: 23.3 ms
Wall time: 30.7 ms

Built In
CPU times: user 2.91 ms, sys: 5.98 ms, total: 8.89 ms
Wall time: 10.2 ms


In [15]:
# Slightly Larger
data = generate_points(10**2, 10)

print('Naive')
%time naive_nn = naive_get_all_nn(data)

print('\nHashed')
%time hashed_nn = hashed_get_all_nn(data)

print('\nBuilt In')
%time built_in_nn = built_in_get_all_nn(data)

Naive
CPU times: user 58.2 ms, sys: 6.14 ms, total: 64.4 ms
Wall time: 71 ms

Hashed
CPU times: user 1.08 ms, sys: 214 µs, total: 1.3 ms
Wall time: 1.56 ms

Built In
CPU times: user 2.74 ms, sys: 1.56 ms, total: 4.3 ms
Wall time: 5.05 ms


In [16]:
# Make sure they all work
data = generate_points(10**3, 10)

print('Naive')
%time naive_nn = naive_get_all_nn(data)

Naive
CPU times: user 4.1 s, sys: 73.5 ms, total: 4.17 s
Wall time: 5.45 s


In [17]:
print('Hashed')
%time hashed_nn = hashed_get_all_nn(data)

Hashed
CPU times: user 8.89 ms, sys: 2.7 ms, total: 11.6 ms
Wall time: 12.8 ms


In [18]:
print('Built In')
%time built_in_nn = built_in_get_all_nn(data)

Built In
CPU times: user 18.7 ms, sys: 3.43 ms, total: 22.1 ms
Wall time: 20.7 ms


In [23]:
# Let's try bigger data
data = generate_points(10**4, 10)

In [24]:
print('Naive')
%time naive_nn = naive_get_all_nn(data)

Naive
CPU times: user 7min 5s, sys: 7.77 s, total: 7min 13s
Wall time: 7min 53s


In [20]:
print('Hashed')
%time hashed_nn = hashed_get_all_nn(data)

Hashed
CPU times: user 9.27 ms, sys: 220 µs, total: 9.49 ms
Wall time: 9.75 ms


In [21]:
print('Built In')
%time built_in_nn = built_in_get_all_nn(data)

Built In
CPU times: user 26.8 ms, sys: 5.79 ms, total: 32.6 ms
Wall time: 31.8 ms
